##Dataset

In [2]:
import gdown
import os
import pandas as pd
from google.colab import drive
# part 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
## برای سوال دوم
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import pickle
import gc
##  بخش سوم
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
file_urls = {
    "train_data.csv": "https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW",
    "test_data.csv": "https://drive.google.com/uc?id=1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB",
    "product_mapping.csv": "https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h"
}

drive.mount('/content/drive')

shared_dir = "/content/drive/MyDrive/Bootcamp"
os.makedirs(shared_dir, exist_ok=True)

for file_name, url in file_urls.items():
    output_path = os.path.join(shared_dir, file_name)
    gdown.download(url, output_path, quiet=False)
    print(f"فایل {file_name} در {shared_dir} ذخیره شد.")

notebook_name = "project2.ipynb"
notebook_path = os.path.join(shared_dir, notebook_name)
print(f"\nنوت‌بوک در {notebook_path} ذخیره خواهد شد.")

print("\nفایل‌های موجود در پوشه مشترک:")
for file in os.listdir(shared_dir):
    if file.endswith((".csv", ".ipynb")):
        print(file)

Mounted at /content/drive


Downloading...
From (original): https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW
From (redirected): https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW&confirm=t&uuid=5d2a202a-11e6-426b-a985-22c59510c956
To: /content/drive/MyDrive/Bootcamp/train_data.csv
100%|██████████| 635M/635M [00:11<00:00, 57.2MB/s]


فایل train_data.csv در /content/drive/MyDrive/Bootcamp ذخیره شد.


Downloading...
From: https://drive.google.com/uc?id=1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB
To: /content/drive/MyDrive/Bootcamp/test_data.csv
100%|██████████| 15.6M/15.6M [00:00<00:00, 40.8MB/s]


فایل test_data.csv در /content/drive/MyDrive/Bootcamp ذخیره شد.


Downloading...
From: https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h
To: /content/drive/MyDrive/Bootcamp/product_mapping.csv
100%|██████████| 97.3M/97.3M [00:00<00:00, 126MB/s]

فایل product_mapping.csv در /content/drive/MyDrive/Bootcamp ذخیره شد.

نوت‌بوک در /content/drive/MyDrive/Bootcamp/project2.ipynb ذخیره خواهد شد.

فایل‌های موجود در پوشه مشترک:
updated_label_data.csv
bert_labeled_data.csv
quick_warranty_sentiment.csv
test_data.csv
train_data.csv
product_mapping.csv


In [ ]:
print("\nنمونه‌ای از train_data.csv:")
df_train = pd.read_csv(os.path.join(shared_dir, "train_data.csv"), nrows=5)
print(df_train.head())


نمونه‌ای از train_data.csv:
   overall  vote  verified  reviewTime      reviewerID       asin  style  \
0        2   NaN     False  2016-11-11   A2OSUEZJIN7BI  511189877    NaN   
1        5   NaN      True  2016-06-06  A2NETQRG6JHIG7  511189877    NaN   
2        4   NaN      True  2016-03-10  A12JHGROAX49G7  511189877    NaN   
3        5   NaN      True  2016-01-14  A1KV65E2TMMG6F  511189877    NaN   
4        5   NaN      True  2016-10-20  A280POPEWI0NSA  594459451    NaN   

    reviewerName                                         reviewText  \
0          Chris  I have an older URC-WR7 remote and thought thi...   
1   Qrysta White  First time I've EVER had a remote that needed ...   
2        Linwood  Got them and only 2 of them worked. company ca...   
3  Dane Williams  I got tired of the remote being on the wrong s...   
4    Kristina H.  After purchasing cheap cords from another webs...   

                               summary  unixReviewTime  
0                         Cann

##Part 2

In [ ]:
!ls -lh

total 1.3G
-rw-r--r-- 1 root root 1.3G Jan 18  2019 cc.en.300.vec.gz
drwxr-xr-x 1 root root 4.0K Jul 10 13:35 sample_data


**Build Gold list of words**

In [ ]:
gold_keywords = {
    # Base words
    "warranty", "guarantee",

    # Similar words
    "replacement", "refund", "return", "repair", "coverage", "support", "assurance",
    "reimbursement", "exchange", "restock", "restocking", "renewal",

    # Support & Services
    "customer service", "after-sales", "service", "claim", "service center", "technical support",
    "support team", "return policy", "money back", "policy", "terms",

    # Commonly used compound words
    "limited warranty", "extended warranty", "manufacturer warranty", "lifetime warranty",
    "repair service", "free return", "return window", "guarantee period", "valid warranty",

    # Commonly typos
    "waranty", "warrenty", "guarentee", "guaranty", "warenty", "guarantie",

    # Informal or slang terms
    "they replaced it", "they refunded me", "got a replacement", "no questions asked",
    "got my money back", "got it fixed", "they fixed it", "they sent me a new one"
}


**Loading fast text**

In [ ]:
from gensim.models import KeyedVectors

# Load the FastText model
fasttext_model = KeyedVectors.load_word2vec_format('cc.en.300.vec.gz', binary=False)

**Find words similar to warranty and guarantee**

In [ ]:
# Define base words
base_words = ['warranty', 'guarantee']

# Find similar words
similar_words = set()
for word in base_words:
    similar = fasttext_model.most_similar(word, topn=20)
    similar_words.update([w for w, score in similar])

# Concoct similar words & base words & typos
#manual_typos = ['waranty', 'guarentee', 'guaranty', 'warrenty']
gold_keywords = {
    # Base words
    "warranty", "guarantee",

    # Similar words
    "replacement", "refund", "return", "repair", "coverage", "support", "assurance",
    "reimbursement", "exchange", "restock", "restocking", "renewal",

    # Support & Services
    "customer service", "after-sales", "service", "claim", "service center", "technical support",
    "support team", "return policy", "money back", "policy", "terms",

    # Commonly used compound words
    "limited warranty", "extended warranty", "manufacturer warranty", "lifetime warranty",
    "repair service", "free return", "return window", "guarantee period", "valid warranty",

    # Commonly typos
    "waranty", "warrenty", "guarentee", "guaranty", "warenty", "guarantie",

    # Informal or slang terms
    "they replaced it", "they refunded me", "got a replacement", "no questions asked",
    "got my money back", "got it fixed", "they fixed it", "they sent me a new one"
}
all_keywords = list(list(similar_words) + list(gold_keywords))

# Show part of the final words
print("کلمات کلیدی نهایی:", list(all_keywords)[:20])

کلمات کلیدی نهایی: ['guarante', 'warranty.What', 'arranty', 'guarantees', 'guarantee.If', 'warantee', 'warranty.For', 'warranty.This', 'assure', 'guarrantee', 'warranties', 'guaranteeing', 'guranteed', 'guarantee.The', 'arantee', 'warranty.I', 'warranty.If', 'guaranteee', 'gurantee', 'warrranty']


**Cleaning dataset & filter comments**

In [ ]:
df = pd.read_csv(os.path.join(shared_dir, "train_data.csv"))
print((df.shape))

/tmp/ipython-input-11-1493105635.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(shared_dir, "train_data.csv"))


(838944, 11)


In [ ]:
# Preprocessing function
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text.split()

# Sampeling dataset
df_sampel = df.sample(n=50000, random_state=42)

# Tokenizeig data
df_sampel['tokens'] = df_sampel['reviewText'].fillna('').apply(preprocess)

# Checking if keywords are inside the comment
def contains_keywords(tokens, keywords):
    return any(word in keywords for word in tokens)

df_sampel['is_warranty_related'] = df_sampel['tokens'].apply(lambda x: contains_keywords(x, all_keywords))

**Calc mean for ASIN**

In [ ]:
# Filter comments
warranty_df = df_sampel[df_sampel['is_warranty_related']]

# Calc mean for ASIN
avg_scores = warranty_df.groupby('asin')['overall'].mean().reset_index()
avg_scores.columns = ['asin', 'avg_warranty_rating']

# Show result
avg_scores.head(30)

,asin,avg_warranty_rating
0,0594481902,2.000000
1,8918010656,3.000000
2,980035977X,1.000000
3,B000001OM4,4.000000
4,B00000K2YR,1.000000
5,B00001P4ZH,5.000000
6,B00001P505,4.500000
7,B00001QEMF,2.000000
8,B00001WRSJ,1.000000
9,B00004SABB,2.000000


## Sentiment Analysis of Customer Reviews on Product Warranty using BERT Models

Analyze customer reviews about product warranties using NLP models, particularly BERT, to classify reviews as "warranty-related" or "not warranty-related."

## What is BERT?
**BERT (Bidirectional Encoder Representations from Transformers):**  
A deep learning model by Google for NLP tasks.  
- **Bidirectional Processing:** Uses both past and future context for better understanding.
- **Various NLP Tasks:** Suitable for classification, named entity recognition, etc.
- **Pre-trained on Large Datasets:** Trained on extensive datasets like books and articles.

## Method 1 - BERT 1.0 --> Using Embeddings and Cosine Similarity

- **Method:** Use BERT embeddings for "warranty" and "guarantee" and calculate cosine similarity with similar words.
- **Advantages:** Identifies similar words and handles spelling errors.
- **Result:** Extracted related words and filtered warranty-related reviews.


In [4]:
import nltk
nltk.download('words')

import os
import torch
import numpy as np
import pandas as pd
from nltk.corpus import words
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

shared_dir = "/content/drive/MyDrive/Bootcamp"
train_path = os.path.join(shared_dir, 'train_data.csv')
mapping_path = os.path.join(shared_dir, 'product_mapping.csv')

df = pd.read_csv(train_path)
mapping_df = pd.read_csv(mapping_path)

df = df[['reviewText', 'overall', 'asin']].dropna(subset=['reviewText'])
df['reviewText'] = df['reviewText'].str.lower()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def get_embedding(word):
    inputs = tokenizer(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

base_words = ['warranty', 'guarantee']
base_embs = np.vstack([get_embedding(w) for w in base_words])

candidate_words = words.words()[:5000]
candidate_words.extend([    # Base words
    "warranty", "guarantee",

    # Similar words
    "replacement", "refund", "return", "repair", "coverage", "support", "assurance",
    "reimbursement", "exchange", "restock", "restocking", "renewal",

    # Support & Services
    "customer service", "after-sales", "service", "claim", "service center", "technical support",
    "support team", "return policy", "money back", "policy", "terms",

    # Commonly used compound words
    "limited warranty", "extended warranty", "manufacturer warranty", "lifetime warranty",
    "repair service", "free return", "return window", "guarantee period", "valid warranty",

    # Commonly typos
    "waranty", "warrenty", "guarentee", "guaranty", "warenty", "guarantie",

    # Informal or slang terms
    "they replaced it", "they refunded me", "got a replacement", "no questions asked",
    "got my money back", "got it fixed", "they fixed it", "they sent me a new one"])

candidate_words = list(set(candidate_words))

candidate_embs = np.vstack([get_embedding(w) for w in candidate_words])

similarities = cosine_similarity(candidate_embs, base_embs).mean(axis=1)

threshold = 0.7
similar_words = [candidate_words[i] for i in range(len(candidate_words)) if similarities[i] > threshold]
print("کلمات مشابه یافت‌شده:", similar_words)

def has_related_word(text, words_list):
    return any(word in text.split() for word in words_list)
df['related_to_warranty'] = df['reviewText'].apply(lambda x: has_related_word(x, similar_words))
filtered_df = df[df['related_to_warranty']]

print("تعداد نظرات مرتبط:", len(filtered_df))

product_satisfaction = filtered_df.groupby('asin')['overall'].mean().reset_index()
product_satisfaction.columns = ['asin', 'avg_satisfaction']

product_satisfaction = product_satisfaction.merge(mapping_df, on='asin', how='left')

product_satisfaction = product_satisfaction.sort_values('avg_satisfaction', ascending=False)
product_satisfaction = product_satisfaction[['asin', 'brand', 'title', 'avg_satisfaction']]

print("جدول رضایت از ضمانت:")
print(product_satisfaction.head(20))

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
/tmp/ipython-input-4-2869257264.py:16: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(train_path)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

کلمات مشابه یافت‌شده: ['aggregation', 'aerator', 'aerobiotic', 'repair', 'acceptive', 'Al', 'affluentness', 'adventuress', 'abbey', 'acronymic', 'addict', 'alanyl', 'agar', 'Aeolism', 'additionally', 'aftosa', 'actual', 'adorableness', 'aim', 'accompany', 'ablation', 'acclivity', 'agria', 'Alaska', 'age', 'adjustive', 'actinian', 'aerial', 'aftergrowth', 'advocate', 'accessory', 'aflare', 'alan', 'ajoint', 'abduction', 'Agrapha', 'acuation', 'lifetime warranty', 'aeronautics', 'abu', 'afterband', 'Acer', 'aftercare', 'affectible', 'adless', 'air', 'academial', 'repair service', 'abacist', 'abir', 'policy', 'warrenty', 'acidproof', 'acrestaff', 'aerate', 'Albert', 'replacement', 'agnel', 'acclaim', 'acceleratory', 'afteract', 'affectedly', 'actionary', 'actualization', 'aftertime', 'abatised', 'adamantoma', 'absolute', 'agrarianize', 'Adamical', 'airing', 'abstractively', 'acquaintance', 'aching', 'accredited', 'afterhend', 'adequately', 'absence', 'advise', 'administerd', 'absorbency',

## Method 2  --> Using Zero-Shot Classification with BART

- **Method:** Use the BART model for zero-shot classification to categorize reviews into "warranty-related" and "not warranty-related."
- **Advantages:** Fast classification with pre-trained models, no need for domain-specific training.
- **Result:** Classified reviews accurately into the two categories.

##- BERT 2.0

In [ ]:
from transformers import pipeline
import pandas as pd

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

df = pd.read_csv('/content/drive/MyDrive/Bootcamp/train_data.csv')
df = df[['reviewText', 'overall', 'asin']].dropna()

candidate_labels = ["warranty-related", "not warranty-related"]

def classify(text):
    result = classifier(text, candidate_labels)
    return result['labels'][0], result['scores'][0]

df_sample = df.head(1000)
df_sample[['predicted_label', 'score']] = df_sample['reviewText'].apply(lambda x: pd.Series(classify(x)))

df_warranty = df_sample[df_sample['predicted_label'] == "warranty-related"]

result = df_warranty.groupby('asin')['overall'].mean().reset_index()
result.columns = ['asin', 'avg_warranty_rating']
print(result.head(20))

result.to_csv('/content/drive/MyDrive/Bootcamp/quick_warranty_sentiment.csv', index=False)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-4-1225161128.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Bootcamp/train_data.csv')
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


          asin  avg_warranty_rating
0   0511189877             4.000000
1   0594296420             5.000000
2   0594481902             3.000000
3   059449771X             5.000000
4   0764207474             5.000000
5   0972683275             5.000000
6   106171327X             5.000000
7   140053271X             4.000000
8   1495443043             5.000000
9   1935009311             5.000000
10  6073894996             1.500000
11  7507825604             4.000000
12  7562434166             3.000000
13  8866139874             1.000000
14  9573212919             1.000000
15  980035977X             3.666667
16  9803751263             1.000000
17  9966338926             2.000000
18  B000001OM4             4.666667
19  B000001OM5             5.000000


/tmp/ipython-input-4-1225161128.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample[['predicted_label', 'score']] = df_sample['reviewText'].apply(lambda x: pd.Series(classify(x)))
/tmp/ipython-input-4-1225161128.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample[['predicted_label', 'score']] = df_sample['reviewText'].apply(lambda x: pd.Series(classify(x)))


##- BERT 2.1

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import re

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

keywords = ['warranty', 'guarantee']

candidate_vocab = [
    # Base words
    "warranty", "guarantee",

    # Similar words
    "replacement", "refund", "return", "repair", "coverage", "support", "assurance",
    "reimbursement", "exchange", "restock", "restocking", "renewal",

    # Support & Services
    "customer service", "after-sales", "service", "claim", "service center", "technical support",
    "support team", "return policy", "money back", "policy", "terms",

    # Commonly used compound words
    "limited warranty", "extended warranty", "manufacturer warranty", "lifetime warranty",
    "repair service", "free return", "return window", "guarantee period", "valid warranty",

    # Commonly typos
    "waranty", "warrenty", "guarentee", "guaranty", "warenty", "guarantie",

    # Informal or slang terms
    "they replaced it", "they refunded me", "got a replacement", "no questions asked",
    "got my money back", "got it fixed", "they fixed it", "they sent me a new one"
]

emb_keywords = embedding_model.encode(keywords, convert_to_tensor=True)
emb_vocab = embedding_model.encode(candidate_vocab, convert_to_tensor=True)
cosine_scores = util.cos_sim(emb_keywords, emb_vocab)

similar_words = set()
for i in range(len(keywords)):
    for j in range(len(candidate_vocab)):
        if cosine_scores[i][j] > 0.6:
            similar_words.add(candidate_vocab[j])

typos = ['waranty', 'warrenty', 'guarentee', 'garantee', 'warrante', 'guaranty', 'warrantee']
similar_keywords = list(set(list(similar_words) + keywords + typos))

df = pd.read_csv('/content/drive/MyDrive/Bootcamp/train_data.csv')
df = df[['reviewText', 'asin', 'overall']].dropna().head(1000)

def has_related_word(text, words_list):
    pattern = r'\b(?:' + '|'.join(re.escape(w) for w in words_list) + r')\b'
    return bool(re.search(pattern, text))

df['related_to_warranty'] = df['reviewText'].apply(lambda x: has_related_word(str(x).lower(), similar_keywords))
df_warranty = df[df['related_to_warranty']]

candidate_labels = ["warranty-related", "not warranty-related"]

def classify(text):
    try:
        result = classifier(text, candidate_labels)
        return result['labels'][0], result['scores'][0]
    except:
        return "error", 0.0

df_warranty[['predicted_label', 'score']] = df_warranty['reviewText'].apply(lambda x: pd.Series(classify(x)))
df_warranty_filtered = df_warranty[df_warranty['predicted_label'] == "warranty-related"]

product_satisfaction = df_warranty_filtered.groupby('asin')['overall'].mean().reset_index()
product_satisfaction.columns = ['asin', 'avg_warranty_rating']

print("First 10 Products with Warranty Satisfaction:")
print(product_satisfaction.head(10))

Device set to use cuda:0
/tmp/ipython-input-14-719488626.py:31: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Bootcamp/train_data.csv')


First 10 Products with Warranty Satisfaction:
         asin  avg_warranty_rating
0  B000001OM4               4.0000
1  B00000J4EY               3.0000
2  B00000K2YR               1.0000
3  B00001P4XA               5.0000
4  B00001P4ZH               4.3125
5  B00001P505               5.0000
6  B00002EQCS               3.0000
7  B00002EQCW               4.0000
8  B00004SABB               1.0000
9  B00004SABJ               4.0000


/tmp/ipython-input-14-719488626.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_warranty[['predicted_label', 'score']] = df_warranty['reviewText'].apply(lambda x: pd.Series(classify(x)))
/tmp/ipython-input-14-719488626.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_warranty[['predicted_label', 'score']] = df_warranty['reviewText'].apply(lambda x: pd.Series(classify(x)))


## Method 3 - BERT 3.0 --> Combining Zero-Shot Classification with Warranty Keywords

- **Method:** Filter reviews based on keywords (e.g., "warranty" and "guarantee") and apply BART classification.
- **Advantages:** Combines keyword matching with advanced classification for better accuracy.
- **Result:** Identified warranty-related reviews and calculated average ratings.


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm

model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv('/content/drive/MyDrive/Bootcamp/train_data.csv')
df = df[['reviewText', 'asin', 'overall']].dropna().head(1000)

warranty_keywords = [
    # Base words
    "warranty", "guarantee",

    # Similar words
    "replacement", "refund", "return", "repair", "coverage", "support", "assurance",
    "reimbursement", "exchange", "restock", "restocking", "renewal",

    # Support & Services
    "customer service", "after-sales", "service", "claim", "service center", "technical support",
    "support team", "return policy", "money back", "policy", "terms",

    # Commonly used compound words
    "limited warranty", "extended warranty", "manufacturer warranty", "lifetime warranty",
    "repair service", "free return", "return window", "guarantee period", "valid warranty",

    # Commonly typos
    "waranty", "warrenty", "guarentee", "guaranty", "warenty", "guarantie",

    # Informal or slang terms
    "they replaced it", "they refunded me", "got a replacement", "no questions asked",
    "got my money back", "got it fixed", "they fixed it", "they sent me a new one"
]

def contains_keywords(text):
    return any(word in text.lower() for word in warranty_keywords)

df_warranty = df[df['reviewText'].apply(contains_keywords)].reset_index(drop=True)

labels = ["warranty-related", "not warranty-related"]
hypothesis_template = "This review is {}."

def zero_shot_batch(texts, labels):
    results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for text in tqdm(texts, desc="Zero-shot inference"):
        inputs = tokenizer(
            [text] * len(labels),
            [hypothesis_template.format(label) for label in labels],
            return_tensors="pt",
            truncation=True,
            padding=True
        ).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits

        probs = softmax(logits.detach().cpu().numpy(), axis=1)[:, -1]
        best_idx = probs.argmax()
        best_label = labels[best_idx]
        best_score = probs[best_idx]
        results.append((best_label, float(best_score)))

    return results

results = zero_shot_batch(df_warranty['reviewText'].tolist(), labels)
df_warranty[['predicted_label', 'score']] = results
df_warranty_final = df_warranty[df_warranty['predicted_label'] == 'warranty-related']

product_satisfaction = df_warranty_final.groupby('asin')['overall'].mean().reset_index()
product_satisfaction.columns = ['asin', 'avg_warranty_rating']

print("First 20 Products with Warranty Satisfaction:")
print(product_satisfaction.sort_values('avg_warranty_rating', ascending=False).head(20))

/tmp/ipython-input-15-2625612796.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Bootcamp/train_data.csv')
Zero-shot inference: 100%|██████████| 83/83 [00:11<00:00,  7.53it/s]

First 20 Products with Warranty Satisfaction:
          asin  avg_warranty_rating
1   0594296420             5.000000
4   1616825375             5.000000
3   059449771X             5.000000
13  B00000K135             5.000000
23  B00004RG7X             5.000000
12  B00000JD3O             5.000000
20  B00001WRSJ             4.857143
18  B00001P505             4.750000
17  B00001P4ZH             4.421053
8   B000001OM4             4.000000
0   0511189877             4.000000
22  B00002EQCW             4.000000
26  B00004SY4H             4.000000
16  B00001P4XH             3.666667
15  B00001P4XA             3.500000
9   B000001ON6             3.500000
10  B00000J4EY             3.000000
21  B00002EQCS             3.000000
25  B00004SABJ             2.500000
19  B00001QEMF             2.000000


## Results of Methods

- **Method 1:** Identified warranty-related reviews using embeddings and cosine similarity.
- **Method 2:** Achieved high classification accuracy using BART for zero-shot classification.
- **Method 3:** Combined keyword matching with BART for more precise results.

---

## Comparison of Results

| Method                      | Accuracy     | Speed    | Advantages                              | Disadvantages                        |
|-----------------------------|--------------|----------|-----------------------------------------|--------------------------------------|
| **Method 1 (Embeddings)**    | Medium       | Fast     | Identifies similar words and spelling mistakes | Needs threshold fine-tuning         |
| **Method 2 (Zero-Shot BART)**| High         | Medium   | Accurate classification with pre-trained models | Might not be accurate in some cases |
| **Method 3 (Keywords + BART)**| High         | Slower   | Better accuracy with combined methods  | Slower processing speed             |

---

## Conclusion

- **Best Method:**  
  **Method 2 (Zero-Shot BART)** is the best for its high accuracy and minimal manual adjustments.
- **Additional Insights:**  
  **Method 3** offers higher accuracy but is slower, while **Method 1** is useful for identifying similar words but lacks the precision of the other methods.
